# Newton-Cotes
The goal of this program is to compute a 3rd order Newton-Cotes for $\int_{0}^{1} f(x)dx$, i.e. Newton-Cotes for 4 equally spaced nodes. 

## General Newton-Cotes
Given a < b, let $x_i$ = a + di (i = 0, 1, ..., n) where $d = \frac{b-a}{n}$ so $x_0, x_1, \ldots, x_n$ are equally spaced nodes on [a,b]. Then, the Newton-Cotes formula of order n (aka degree n) for $\int_{a}^{b} f(x)dx$ is given by
$\int_{a}^{b} f(x)dx \approx \sum_{i=0}^{n}A_if(x_i)$ where $A_i = \int_{a}^{b} l_i(x)dx$ and $l_i(x) = \prod_{i=0,j\neq i}^{n} \frac{x-x_j}{x_i-x_j}$

### Applying to our problem
The problem tells us that n=3, a=0, and b=1. So 
$\int_{0}^{1} f(x)dx \approx \sum_{i=0}^{3} A_if(x_i)$ where $A_i = \int_{0}^{1} l_i(x)dx$, $l_i(x) = \prod_{i=0,j\neq i}^{3} \frac{x-x_j}{x_i-x_j}$, and $\{x_i\}_{0 \leq i \leq 3}$ = 0, $\frac{1}{3}$, $\frac{2}{3}$, 1 

In [3]:
import numpy as np
from scipy.integrate import quad 

def newtonCotes(f):
    '''
    Input data: 
    * the function (f) we're approximating the integral for
    * assuming the interval for the integral is [0,1] and we have 4 equally spaced nodes
    '''
    nodes = [0,1/3,2/3,1]
    A = []
    for i in range(4):
        l = lambda x: np.prod([(x-nodes[j])/(nodes[i]-nodes[j]) for j in range(4) if j != i])
        integral = quad(l,0,1)
        A.append(integral[0])

    total = sum(A[i]* f(nodes[i]) for i in range(4))
    return total

print(newtonCotes(f = lambda x: x))


0.5


### Generalizing the code
Instead of fixing the number of nodes (n) to three and setting the interval to [0,1], we're now going to consider any interval [a,b] (where a<b) and any number of nodes.

In [5]:
import numpy as np
from scipy.integrate import quad 

def genNewtonCotes(f, a, b, n):
    '''
    Input data:
    * the function (f)
    * the left and right endpoints of the interval (a and b respectively)
    Here we're assuming that a < b
    * the number of nodes (n)
    Here we're assuming that n is an integer
    '''
    d = (b-a)/n
    A = []
    nodes = [(a + d*k) for k in range(n+1)]
    for i in range(n):
        l = lambda x: np.prod([(x-nodes[j])/(nodes[i]-nodes[j]) for j in range(n) if j!= i])
        integral = quad(l,a,b)
        A.append(integral[0])
    total = sum(A[i]*f(nodes[i]) for i in range(n))
    return total

print(genNewtonCotes(f = lambda x:x, a = 0, b= 1, n =3))
        

0.5


# Gausian Quadrature

The goal for the next algorithm is to compute a Gaussian quadrature on the interval [0,1] using two nodes (i.e. n=1).

## General Gaussian Quadrature

Given a < b, let w be some positive weight function (i.e. w:[a,b] $\rightarrow$ (0, $\infty$)). Then for any distinct $x_0, x_1, \dots x_n \in [a,b]$, we have that $\int_{a}^{b} f(x)w(x)dx \approx \sum_{i=0}^n \tilde{A}_if(x_i)$, where $\tilde{A}_i = \int_{a}^{b} l_i(x)w(x)dx$ and $l_i(x) = \prod_{j \neq i} \frac{x-x_j}{x_i-x_j}$.

### Applying to our problem

The problem tells us that we only have two nodes and the interval is [0,1]. So $\int_{0}^{1} f(x)w(x)dx \approx \sum_{i=0}^1 \tilde{A}_if(x_i)$ where $\tilde{A}_i = \int_{0}^{1} l_i(x)w(x)dx$.

In [13]:
import numpy as np
from scipy.integrate import quad

def gaussian(f, w):
    '''
    Inputs:
    * the function (f)
    * the weight function (w)
    Output:
    * integral approximation
    '''
    nodes = [0,1]
    A_tilda = []
    for i in range(2):
        l = lambda x: np.prod([(x-nodes[j])/(nodes[i]-nodes[j]) for j in range(2) if j != i]) * w(x)
        integral = quad(l,0,1)
        A_tilda.append(integral[0])
    total = A_tilda[0]*f(nodes[0]) + A_tilda[1]*f(nodes[1])
    return total

print(gaussian(f = lambda x: x,w = lambda x: x**2 + 1))
    

0.7499999999999999


### Generalizing the code

Instead of fixing the number of nodes (n) and choosing the interval to be [0,1], we're going to generalize the code for any interval [a,b] (assuming a <b) and any number of nodes (note that the nodes do not have to be evenly spaced like Newton-Cotes)

In [15]:
import numpy as np
from scipy.integrate import quad

def genGaussian(f, w, nodes, a, b):
    '''
    Inputs:
    * the function (f) and the positive weight function (w)
    * the nodes 
    * the interval for the integral (a, b) assuming that a < b
    Output:
    * integral approximation for f(x)*w(x)
    '''
    A = []
    for i in range(len(nodes)):
        l = lambda x: np.prod([(x-nodes[j])/(nodes[i]-nodes[j]) for j in range(len(nodes)) if j != i]) * w(x)
        integral = quad(l,a,b)
        A.append(integral[0])
    total = sum(A[i]*f(nodes[i]) for i in range(len(nodes)))
    return total

print(genGaussian(f = lambda x:x, w = lambda x: x**2 + 1, nodes = [0,1/3,2/3,1], a = 0, b = 1))


0.75


## Gaussian-Legendre2 Quadrature

When using the Gaussian quadrature formula, when w(x) $\equiv$ 1, this is called a Gaussian-Legendre quadrature (aka **the** Gaussian quadrature). For the Gaussian-Legendre2 Quadrature, our nodes are $\frac{-1}{\sqrt{3}}, \frac{1}{\sqrt{3}}$ for our interval [-1,1].

In [ ]:
import numpy as np

def legendre2(f, a: float, b: float):
    '''                  
    Inputs:
    * function (f)
    * end points of the interval (a, b)
    Output:
    * Gaussian-Legendre2 quadrature for the integral approximation
    '''
    tmp = 1/np.sqrt(3)
    nodes = np.array([-tmp, tmp])
    weights = np.array([-tmp,tmp])
    left_pt = a
    x = left_pt + (b-a)*nodes # broadcasting in effect
    height = f(x)
    [height_left,heigh_right] = height
    return weights[0]*height_left + weights[1]*heigh_right
